In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [85]:
pre_df = pd.read_csv('../data/cleaned/cleaned_vessel_calls.csv')
pre_df = pre_df.rename(columns={'arrivalDate':'arrival_date', 'sailingDate':'sailing_date', 'place.unlocode':'current_unlocode', 'place.placeName':'current_port','vessel.vesselType':'vessel_type', 'place.placeType': 'place_type', 'place.countryName': 'country_name', 'vessel.vesselName': 'vessel_name'})

In [86]:
# preprocess data
# sort by imo number
pre_df.sort_values(by=["IMO", "arrival_date"], inplace=True)
pre_df['next_unlocode'] = pre_df.groupby('IMO')['current_unlocode'].shift(-1)
pre_df['is_last_imo'] = pre_df['IMO'] != pre_df['IMO'].shift(-1)
pre_df = pre_df[~pre_df["is_last_imo"]]
pre_df = pre_df[pre_df["Event_Type"] != "ARRIVAL"]

In [87]:
pre_df = pre_df.drop(columns=["is_last_imo", "Event_Type", "Timestamp", 'vessel_name'])
pre_df

,arrival_date,sailing_date,current_port,current_unlocode,place_type,country_name,IMO,vessel_type,next_unlocode
314761,2023-01-18T14:10:05,2023-01-26T04:05:00,Los Angeles,USLAX,Port,United States of America,1026764,Vessel Type (unspecified),CNJGY
314763,2023-02-22T07:17:58,2023-02-23T18:56:16,Jiangyin,CNJGY,Port,People's Republic of China,1026764,Vessel Type (unspecified),CNTXG
314765,2023-03-02T16:00:39,2023-03-03T12:31:09,Tianjin Xingang Port,CNTXG,Port,People's Republic of China,1026764,Vessel Type (unspecified),CNQAW
314767,2023-03-04T21:02:13,2023-03-05T11:45:29,Qianwan,CNQAW,Sub Port,People's Republic of China,1026764,Vessel Type (unspecified),CNMSN
314769,2023-03-06T23:30:18,2023-03-08T00:00:46,Meishan,CNMSN,Sub Port,People's Republic of China,1026764,Vessel Type (unspecified),CNNSA
...,...,...,...,...,...,...,...,...,...
12569,2023-09-05T05:50:48,2023-09-06T02:46:03,Port Klang (Pelabuhan Klang),MYPKG,Port,Malaysia,9951135,fully cellular containership,INMAA
12571,2023-09-10T16:43:35,2023-09-11T20:02:20,Chennai (ex Madras),INMAA,Port,India,9951135,fully cellular containership,BDCGP
12573,2023-09-15T05:31:56,2023-09-17T05:28:29,Chittagong (Chattogram),BDCGP,Port,Bangladesh,9951135,fully cellular containership,THLCH
12575,2023-09-24T15:15:36,2023-09-25T08:23:38,Laem Chabang,THLCH,Port,Thailand,9951135,fully cellular containership,SGSIN


In [88]:
# grab all of the ports that currently have singapore as the current port
current_singapore_df = pre_df[pre_df['current_unlocode'] == 'SGSIN']
next_singapore_df = pre_df[pre_df['next_unlocode'] == 'SGSIN']

In [89]:
current_singapore_df = current_singapore_df.drop(columns=['current_unlocode', 'current_port', 'place_type', 'country_name'])
next_singapore_df = next_singapore_df.drop(columns=['next_unlocode'])

In [90]:
# add features from next_singapore_df to current_singapore_df, like country_name and place_type
merge_next_df = next_singapore_df[['current_unlocode', 'country_name', 'place_type']]
merge_next_df = merge_next_df.rename(columns={'current_unlocode': 'next_unlocode', 'country_name': 'next_country_name', 'place_type': 'next_place_type'})

current_singapore_df = current_singapore_df.merge(merge_next_df, on='next_unlocode', how='left')
# drop duplicates
current_singapore_df = current_singapore_df.drop_duplicates()
current_singapore_df

,arrival_date,sailing_date,IMO,vessel_type,next_unlocode,next_country_name,next_place_type
0,2023-03-17T16:15:03,2023-03-18T08:30:03,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
157,2023-05-19T09:10:09,2023-05-20T12:20:45,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
314,2023-07-23T01:10:02,2023-07-23T20:45:01,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
471,2021-08-13T15:20:03,2021-08-14T01:40:07,7433880,tug/supply,TWTPE,"Taiwan, China",Port
542,2021-11-06T05:15:03,2021-11-06T18:55:01,7433880,tug/supply,IDTPP,Indonesia,Port
...,...,...,...,...,...,...,...
2116888,2023-06-16T05:01:40,2023-06-18T06:53:29,9943322,Combined chemical and oil tanker,USLGB,United States of America,Port
2116905,2023-08-01T00:13:39,2023-08-01T13:43:43,9951135,fully cellular containership,MYPKG,Malaysia,Port
2117450,2023-08-26T06:09:29,2023-08-26T13:49:33,9951135,fully cellular containership,THLCH,Thailand,Port
2117850,2023-09-03T23:34:09,2023-09-04T11:03:17,9951135,fully cellular containership,MYPKG,Malaysia,Port


In [91]:
# print the null values in the next_country_name and next_place_type columns
temp_df = (current_singapore_df[current_singapore_df['next_country_name'].isnull()]['next_unlocode'].value_counts())
temp_df

next_unlocode
GHTEM    56
TRDRC    34
BRRIO    26
EGPSD    13
ILASH    11
         ..
AUNHU     1
PHTGO     1
IDPSS     1
USMAH     1
CABUB     1
Name: count, Length: 127, dtype: int64

In [92]:
# print the null values in the next_country_name and next_place_type columns
temp_df = (current_singapore_df[current_singapore_df['next_place_type'].isnull()]['next_unlocode'].value_counts())
temp_df

next_unlocode
GHTEM    56
TRDRC    34
BRRIO    26
EGPSD    13
ILASH    11
         ..
AUNHU     1
PHTGO     1
IDPSS     1
USMAH     1
CABUB     1
Name: count, Length: 127, dtype: int64

In [93]:
# # let's fix the null values in the next_country_name and next_place_type columns - we can do this by looking at the current_unlocode
# # and filling in the values
current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'GHTEM', 'next_country_name'] = 'Ghana'
current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'GHTEM', 'next_place_type'] = 'Port'

current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'TRDRC', 'next_country_name'] = 'Turkey'
current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'TRDRC', 'next_place_type'] = 'Port'

current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'BRRIO', 'next_country_name'] = 'Brazil'
current_singapore_df.loc[current_singapore_df['next_unlocode'] == 'BRRIO', 'next_place_type'] = 'Port'

In [94]:
# drop null values
current_singapore_df = current_singapore_df.dropna()
current_singapore_df

,arrival_date,sailing_date,IMO,vessel_type,next_unlocode,next_country_name,next_place_type
0,2023-03-17T16:15:03,2023-03-18T08:30:03,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
157,2023-05-19T09:10:09,2023-05-20T12:20:45,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
314,2023-07-23T01:10:02,2023-07-23T20:45:01,1026764,Vessel Type (unspecified),SAJED,Saudi Arabia,Port
471,2021-08-13T15:20:03,2021-08-14T01:40:07,7433880,tug/supply,TWTPE,"Taiwan, China",Port
542,2021-11-06T05:15:03,2021-11-06T18:55:01,7433880,tug/supply,IDTPP,Indonesia,Port
...,...,...,...,...,...,...,...
2116888,2023-06-16T05:01:40,2023-06-18T06:53:29,9943322,Combined chemical and oil tanker,USLGB,United States of America,Port
2116905,2023-08-01T00:13:39,2023-08-01T13:43:43,9951135,fully cellular containership,MYPKG,Malaysia,Port
2117450,2023-08-26T06:09:29,2023-08-26T13:49:33,9951135,fully cellular containership,THLCH,Thailand,Port
2117850,2023-09-03T23:34:09,2023-09-04T11:03:17,9951135,fully cellular containership,MYPKG,Malaysia,Port


In [97]:
# handle times
# preprocess dataframe - duration and month
current_singapore_df["arrival_date"] = pd.to_datetime(current_singapore_df["arrival_date"])
current_singapore_df["sailing_date"] = pd.to_datetime(current_singapore_df["sailing_date"])
current_singapore_df["duration"] = pd.to_datetime(current_singapore_df["sailing_date"]) - pd.to_datetime(current_singapore_df["arrival_date"])
current_singapore_df["month"] = current_singapore_df["arrival_date"].dt.month

# current_singapore_df['day'] = current_singapore_df['arrivalDate'].dt.day
current_singapore_df['week'] = current_singapore_df['arrival_date'].dt.isocalendar().week
current_singapore_df['quarter'] = current_singapore_df['arrival_date'].dt.quarter

# current_singapore_df['day'] = current_singapore_df['day'].astype(int)
current_singapore_df['week'] = current_singapore_df['week'].astype(int)
current_singapore_df['quarter'] = current_singapore_df['quarter'].astype(int)

C:\Users\syeda\AppData\Local\Temp\ipykernel_24924\3729503970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_singapore_df["arrival_date"] = pd.to_datetime(current_singapore_df["arrival_date"])
C:\Users\syeda\AppData\Local\Temp\ipykernel_24924\3729503970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_singapore_df["sailing_date"] = pd.to_datetime(current_singapore_df["sailing_date"])
C:\Users\syeda\AppData\Local\Temp\ipykernel_24924\3729503970.py:5: SettingWithCopyWarning: 
A value is